In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
import scipy.io
import pandas as pd
import numpy as np

%matplotlib inline


In [ ]:
import scipy.io

EEG = scipy.io.loadmat('/home/mario/code/SeanDominique/will-they-wake-up/data/physionet.org/files/i-care/2.1/training/0360/0360_023_061_EEG.mat')
EEG = EEG['val'][0]

In [ ]:
EEG = EEG.astype(float)
EEG.shape

In [ ]:
import librosa


In [ ]:


tempogram = librosa.stft(y=EEG, n_fft=1024, center=False)


In [ ]:
tempogram.shape

In [ ]:
fig, ax = plt.subplots()

img = librosa.display.specshow(librosa.amplitude_to_db(tempogram,ref=np.max), y_axis='log',x_axis='time',ax=ax)
ax.set_title('Power spectrogram')

fig.colorbar(img, ax=ax, format="%+2.0f dB")

In [ ]:
Fs = 500
fft_result = np.fft.fft(EEG)
freq = np.fft.fftfreq(len(EEG), d=1/Fs)
power_spectrum = np.abs(fft_result)
plt.plot(freq[:len(freq)//2], power_spectrum[:len(power_spectrum)//2])
plt.title("EEG Power Spectrum")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.show()


In [ ]:
from scipy.signal import welch

f, psd = welch(EEG, fs=Fs, nperseg=1024)
plt.semilogy(f, psd)
plt.title("Power Spectral Density")
plt.xlabel("Frequency (Hz)")
plt.ylabel("PSD (V²/Hz)")
plt.show()

In [ ]:
px.line(x=f,y=psd,log_y=True)

In [ ]:
EEG2 = scipy.io.loadmat('/home/mario/code/SeanDominique/will-they-wake-up/data/physionet.org/files/i-care/2.1/training/0284/0284_001_004_EEG.mat')
EEG2 = EEG2['val'][0]
EEG2 = EEG2.astype(float)

In [ ]:
fft_result = np.fft.fft(EEG2)
freq = np.fft.fftfreq(len(EEG2), d=1/Fs)
power_spectrum = np.abs(fft_result)
plt.plot(freq[:len(freq)//2], power_spectrum[:len(power_spectrum)//2])
plt.title("EEG Power Spectrum")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.show()

In [ ]:
f, psd = welch(EEG2, fs=Fs, nperseg=1024)
plt.semilogy(f, psd)
plt.title("Power Spectral Density")
plt.xlabel("Frequency (Hz)")
plt.ylabel("PSD (V²/Hz)")
plt.show()

In [ ]:
px.line(x=f,y=psd,log_y=True)

In [ ]:
EEG3 = scipy.io.loadmat('/home/mario/code/SeanDominique/will-they-wake-up/data/physionet.org/files/i-care/2.1/training/0284/0284_004_007_EEG.mat')
EEG3 = EEG3['val'][0]
EEG3 = EEG3.astype(float)

In [ ]:
f, psd = welch(EEG3, fs=Fs, nperseg=2000)
plt.semilogy(f, psd)
plt.title("Power Spectral Density")
plt.xlabel("Frequency (Hz)")
plt.ylabel("PSD (V²/Hz)")
plt.show()

In [ ]:
px.line(x=f,y=psd,log_y=False)


In [ ]:
fft_result = np.fft.fft(EEG3)
freq = np.fft.fftfreq(len(EEG3), d=1/Fs)
power_spectrum = np.abs(fft_result)
plt.plot(freq[:len(freq)//2], power_spectrum[:len(power_spectrum)//2])
plt.title("EEG Power Spectrum")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.show()

In [ ]:
!pip install mne


In [ ]:
import mne

In [ ]:
!pip install pymatreader

In [ ]:
EEG4 = scipy.io.loadmat('/home/mario/code/SeanDominique/will-they-wake-up/data/physionet.org/files/i-care/2.1/training/0284/0284_017_017_EEG.mat')
EEG4 = EEG4['val'][15]
EEG4 = EEG4.astype(float)

In [ ]:
f, psd = welch(EEG4, fs=Fs, nperseg=2048)
plt.semilogy(f, psd)
plt.title("Power Spectral Density")
plt.xlabel("Frequency (Hz)")
plt.ylabel("PSD (V²/Hz)")
plt.show()

In [ ]:
px.line(x=f,y=psd,log_y=False)
